In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Dataset de ventas para análisis
ventas = pd.DataFrame({
    'venta_id': ['V001', 'V002', 'V003', 'V004', 'V005', 'V006', 'V007', 'V008'],
    'cliente': ['Empresa A', 'Empresa B', 'Empresa C', 'Empresa A', 'Empresa D', 'Empresa B', 'Empresa C', 'Empresa E'],
    'producto': ['Consultoría', 'Software', 'Consultoría', 'Training', 'Software', 'Consultoría', 'Training', 'Software'],
    'cantidad': [1, 2, 1, 3, 1, 2, 2, 1],
    'precio_unitario': [50000, 25000, 75000, 15000, 30000, 60000, 20000, 35000],
    'descuento_pct': [0, 5, 10, 0, 15, 8, 5, 12],
    'fecha_venta': ['2024-01-15', '2024-01-20', '2024-02-10', '2024-02-15', '2024-03-05', '2024-03-10', '2024-03-20', '2024-04-01'],
    'vendedor': ['Ana', 'Carlos', 'María', 'Ana', 'Pedro', 'Carlos', 'María', 'Pedro']
})

# Convertir fecha a datetime
ventas['fecha_venta'] = pd.to_datetime(ventas['fecha_venta'])
print(ventas.info())
print(ventas.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   venta_id         8 non-null      object        
 1   cliente          8 non-null      object        
 2   producto         8 non-null      object        
 3   cantidad         8 non-null      int64         
 4   precio_unitario  8 non-null      int64         
 5   descuento_pct    8 non-null      int64         
 6   fecha_venta      8 non-null      datetime64[ns]
 7   vendedor         8 non-null      object        
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 644.0+ bytes
None
  venta_id    cliente     producto  cantidad  precio_unitario  descuento_pct  \
0     V001  Empresa A  Consultoría         1            50000              0   
1     V002  Empresa B     Software         2            25000              5   
2     V003  Empresa C  Consultoría         1       

In [ ]:
# TODO: Crear las siguientes columnas:
# 1. 'precio_con_descuento': precio unitario después del descuento
# 2. 'categoria_precio': 'Alto' si precio_unitario > 40000, sino 'Normal'
# 3. 'mes_texto': nombre del mes de la venta

# Tu solución aquí:
ventas['precio_con_descuento'] = ventas['precio_unitario'] * (1 - ventas['descuento_pct'] / 100)
ventas['categoria_precio'] = np.where(ventas['precio_unitario'] > 40000, 'Alto', 'Normal')
ventas['mes_texto'] = ventas['fecha_venta'].dt.month_name
ventas[['cantidad','precio_unitario','descuento_pct','fecha_venta','precio_con_descuento','categoria_precio','mes_texto']].head()

,cantidad,precio_unitario,descuento_pct,fecha_venta,precio_con_descuento,categoria_precio,mes_texto
0,1,50000,0,2024-01-15,50000.0,Alto,January
1,2,25000,5,2024-01-20,23750.0,Normal,January
2,1,75000,10,2024-02-10,67500.0,Alto,February
3,3,15000,0,2024-02-15,15000.0,Normal,February
4,1,30000,15,2024-03-05,25500.0,Normal,March


In [17]:
# TODO: Crear columnas que muestren:
# 1. 'trimestre_texto': 'Q1', 'Q2', etc.
# 2. 'es_primer_semestre': True si la venta fue en los primeros 6 meses
# 3. 'semanas_desde_inicio': semanas transcurridas desde la primera venta

# Tu solución aquí:
ventas['trimestre_texto'] = 'Q' + ((ventas['fecha_venta'].dt.month - 1) // 3 + 1).astype(str)
ventas['es_primer_semestre'] = ventas['fecha_venta'].dt.month <= 6
ventas['semanas_desde_inicio'] = ((ventas['fecha_venta'] - ventas['fecha_venta'].min()).dt.days // 7).astype(int)
ventas[['venta_id', 'fecha_venta', 'trimestre_texto', 'es_primer_semestre', 'semanas_desde_inicio']].head()

,venta_id,fecha_venta,trimestre_texto,es_primer_semestre,semanas_desde_inicio
0,V001,2024-01-15,Q1,True,0
1,V002,2024-01-20,Q1,True,0
2,V003,2024-02-10,Q1,True,3
3,V004,2024-02-15,Q1,True,4
4,V005,2024-03-05,Q1,True,7


In [18]:
# TODO: Crear una función que evalúe el 'potencial_cliente':
# - 'Alto': si tiene más de 1 compra Y valor total > 50000
# - 'Medio': si tiene 1 compra Y valor > 30000, O más compras con valor total <= 50000
# - 'Bajo': otros casos

# Tu solución aquí:
# Calcular total neto por venta
ventas_df = ventas.copy()
ventas_df['total_bruto'] = ventas_df['cantidad'] * ventas_df['precio_unitario']
ventas_df['descuento_monto'] = ventas_df['total_bruto'] * (ventas_df['descuento_pct'] / 100)
ventas_df['total_neto'] = ventas_df['total_bruto'] - ventas_df['descuento_monto']

# Agrupar por cliente para obtener métricas
cliente_resumen = ventas_df.groupby('cliente').agg({
    'venta_id': 'count',
    'total_neto': 'sum'
}).rename(columns={'venta_id': 'num_compras', 'total_neto': 'valor_total'}).reset_index()
# Función para clasificar potencial

def clasificar_potencial(row):
    num_compras = row['num_compras']
    valor_total = row['valor_total']
    
    if num_compras > 1 and valor_total > 50000:
        return 'Alto'
    elif (num_compras == 1 and valor_total > 30000) or (num_compras > 1 and valor_total <= 50000):
        return 'Medio'
    else:
        return 'Bajo'

# Aplicar la clasificación
cliente_resumen['potencial_cliente'] = cliente_resumen.apply(clasificar_potencial, axis=1)

print( cliente_resumen)

     cliente  num_compras  valor_total potencial_cliente
0  Empresa A            2      95000.0              Alto
1  Empresa B            2     157900.0              Alto
2  Empresa C            2     105500.0              Alto
3  Empresa D            1      25500.0              Bajo
4  Empresa E            1      30800.0             Medio
